# 03. Data Preprocessing | المعالجة المسبقة للبيانات

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Example 1: Data Loading and Exploration** - Know your data structure
- ✅ **Example 2: Data Cleaning** - Have clean data to preprocess
- ✅ **Basic ML concepts**: Features, targets, training vs testing

**If you haven't completed these**, you might struggle with:
- Understanding why preprocessing is needed
- Knowing which preprocessing method to use
- Understanding the difference between scaling methods

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the THIRD example** - it prepares clean data for machine learning!

**Why this example THIRD?**
- **Before** you can build ML models, you need preprocessed data
- **Before** you can train models, you need scaled features
- **Before** you can make predictions, you need encoded categories

**Builds on**: 
- 📓 Example 1: Data Loading (we know our data)
- 📓 Example 2: Data Cleaning (we have clean data)

**Leads to**: 
- 📓 Example 4: Linear Regression (needs preprocessed data)
- 📓 Example 5: Polynomial Regression (needs scaled features)
- 📓 All ML models (all need preprocessing!)

**Why this order?**
1. Preprocessing transforms **clean data into ML-ready format**
2. Preprocessing teaches you **scaling vs encoding** (different problems, different solutions)
3. Preprocessing shows you **train-test split** (essential for evaluation)

---

## The Story: Preparing Ingredients for Cooking | القصة: تحضير المكونات للطبخ

Imagine you're cooking. **Before** you can cook, you need to prepare ingredients - cut vegetables to the same size, marinate meat, measure spices. **After** preparing everything uniformly, you can cook a perfect meal!

Same with machine learning: **Before** building models, we preprocess data - scale features to same range, encode categories to numbers, split into train/test. **After** preprocessing, we can build accurate models!

---

## Why Data Preprocessing Matters | لماذا تهم المعالجة المسبقة للبيانات؟

Preprocessing is essential for ML success:
- **Feature Scaling**: Algorithms work better when features are on similar scales
- **Encoding**: ML algorithms need numbers, not text categories
- **Train-Test Split**: We need separate data to evaluate model performance
- **Without Preprocessing**: Models fail or perform poorly

## Learning Objectives | أهداف التعلم
1. Scale features using Standardization and Normalization
2. Encode categorical variables (Label vs One-Hot)
3. Split data into training and testing sets
4. Understand when to use each preprocessing method
5. Build a complete preprocessing pipeline

In [1]:
# Step 1: Import necessary libraries
# These libraries help us preprocess data for machine learning

import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
from sklearn.preprocessing import (
    StandardScaler,    # For standardization (mean=0, std=1)
    MinMaxScaler,      # For normalization (range 0-1)
    LabelEncoder,      # For ordinal encoding
    OneHotEncoder      # For nominal encoding
)
from sklearn.model_selection import train_test_split  # For splitting data

print("✅ Libraries imported successfully!")
print("\n📚 What each preprocessing tool does:")
print("   - StandardScaler: Scale features to mean=0, std=1 (z-score)")
print("   - MinMaxScaler: Scale features to range [0, 1]")
print("   - LabelEncoder: Convert categories to numbers (ordinal)")
print("   - OneHotEncoder: Convert categories to binary columns (nominal)")
print("   - train_test_split: Split data into train/test sets")

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


✅ Libraries imported successfully!

📚 What each preprocessing tool does:
   - StandardScaler: Scale features to mean=0, std=1 (z-score)
   - MinMaxScaler: Scale features to range [0, 1]
   - LabelEncoder: Convert categories to numbers (ordinal)
   - OneHotEncoder: Convert categories to binary columns (nominal)
   - train_test_split: Split data into train/test sets


## Part 1: Setting the Scene | الجزء الأول: إعداد المشهد

**BEFORE**: We have clean data, but it's not ready for machine learning - features have different scales, categories are text, and we haven't split the data.

**AFTER**: We'll preprocess the data - scale features, encode categories, split into train/test - making it ready for ML models!

**Why this matters**: Most ML algorithms require preprocessed data. Without it, models fail or give poor results!

In [2]:
# Create sample data with different scales and categories
# This simulates real-world data that needs preprocessing

print("\n1. Creating sample data...")
print("إنشاء بيانات نموذجية...")

np.random.seed(42)  # For reproducibility
data = {
    'age': np.random.randint(20, 60, 100),              # Range: 20-60
    'salary': np.random.randint(30000, 100000, 100),    # Range: 30k-100k (different scale!)
    'experience': np.random.uniform(0, 15, 100),        # Range: 0-15 (different scale!)
    'city': np.random.choice(['Riyadh', 'Jeddah', 'Dammam', 'Khobar'], 100),  # Categorical (text)
    'education': np.random.choice(['Bachelor', 'Master', 'PhD'], 100),        # Categorical (text)
    'target': np.random.choice([0, 1], 100)  # Binary target for classification
}
df = pd.DataFrame(data)

print("\n📊 Original Data:")
print(df.head())
print(f"\n📐 Data Shape: {df.shape}")
print("\n🔍 Notice:")
print("   - Numeric features have VERY different scales (age: 20-60, salary: 30k-100k)")
print("   - Categorical features are text (city, education)")
print("   - ML algorithms need same-scale numbers!")



1. Creating sample data...
إنشاء بيانات نموذجية...

📊 Original Data:
   age  salary  experience    city education  target
0   58   65222    9.899761  Jeddah  Bachelor       0
1   48   93335   12.258333  Dammam    Master       0
2   34   40965    8.328012  Riyadh    Master       0
3   27   54538    7.944759  Jeddah    Master       0
4   40   38110    3.627784  Dammam  Bachelor       1

📐 Data Shape: (100, 6)

🔍 Notice:
   - Numeric features have VERY different scales (age: 20-60, salary: 30k-100k)
   - Categorical features are text (city, education)
   - ML algorithms need same-scale numbers!


## Step 1: Create Sample Data | الخطوة 1: إنشاء بيانات نموذجية

**BEFORE**: We need to learn preprocessing, but we need sample data with different scales and categories.

**AFTER**: We'll create data with numeric features (different scales) and categorical features (text) to practice preprocessing!

**Why create this data?** Real datasets have these characteristics - we need to learn how to handle them.

In [3]:
print("\n" + "=" * 60)
print("2. Feature Scaling - Standardization")
print("قياس الميزات - التوحيد المعياري")
print("=" * 60)



2. Feature Scaling - Standardization
قياس الميزات - التوحيد المعياري


## Step 2: Feature Scaling - Standardization | الخطوة 2: قياس الميزات - التوحيد المعياري

**BEFORE**: Features have very different scales (age: 20-60, salary: 30k-100k). Algorithms will be biased toward larger numbers!

**AFTER**: We'll standardize features so they all have mean=0 and std=1, putting them on the same scale!

**Why Standardization?**
- Formula: `(x - mean) / std`
- Result: Mean = 0, Standard Deviation = 1
- Use when: Data is normally distributed, outliers are important
- Good for: Linear models, neural networks, distance-based algorithms

In [4]:
# Create StandardScaler object
# Standardization formula: (x - mean) / std
# This transforms data so mean=0 and std=1
scaler_standard = StandardScaler()
print("   ✅ StandardScaler created")
print("   Formula: (x - mean) / std")

   ✅ StandardScaler created
   Formula: (x - mean) / std


In [5]:
# Select numeric columns for scaling
# Why only numeric? Categorical columns need encoding, not scaling!
numeric_cols = ['age', 'salary', 'experience']
df_scaled_standard = df.copy()
print(f"   Scaling columns: {numeric_cols}")

   Scaling columns: ['age', 'salary', 'experience']


In [6]:
# Fit and transform the data
# .fit_transform() learns the mean/std from data, then transforms it
# Why fit_transform? We learn parameters from training data, then apply to all data

df_scaled_standard[numeric_cols] = scaler_standard.fit_transform(df[numeric_cols])

print("\n✅ After Standardization (mean=0, std=1):")
print("بعد التوحيد المعياري (متوسط=0، انحراف معياري=1):")
print(f"   Age - Mean: {df_scaled_standard['age'].mean():.4f}, Std: {df_scaled_standard['age'].std():.4f}")
print(f"   Salary - Mean: {df_scaled_standard['salary'].mean():.4f}, Std: {df_scaled_standard['salary'].std():.4f}")
print(f"   Experience - Mean: {df_scaled_standard['experience'].mean():.4f}, Std: {df_scaled_standard['experience'].std():.4f}")
print("\n   ✅ All features now have mean≈0 and std≈1!")
print("\n📄 Scaled data (first 5 rows):")
print(df_scaled_standard[numeric_cols].head())


✅ After Standardization (mean=0, std=1):
بعد التوحيد المعياري (متوسط=0، انحراف معياري=1):
   Age - Mean: 0.0000, Std: 1.0050
   Salary - Mean: 0.0000, Std: 1.0050
   Experience - Mean: 0.0000, Std: 1.0050

   ✅ All features now have mean≈0 and std≈1!

📄 Scaled data (first 5 rows):
        age    salary  experience
0  1.670713 -0.057025    0.366648
1  0.801003  1.290892    0.918449
2 -0.416591 -1.220060   -0.001071
3 -1.025388 -0.569284   -0.090735
4  0.105235 -1.356947   -1.100715


## Step 3: Feature Scaling - Normalization (Min-Max) | الخطوة 3: قياس الميزات - التطبيع

**BEFORE**: Features have different ranges. We want them all in the same range [0, 1].

**AFTER**: We'll normalize features so they all range from 0 to 1!

**Why Normalization (Min-Max)?**
- Formula: `(x - min) / (max - min)`
- Result: Range [0, 1]
- Use when: Data is not normally distributed, you want bounded values
- Good for: Neural networks, algorithms that need [0,1] range

In [7]:
# Normalization: (x - min) / (max - min) -> range [0, 1]
# التطبيع: (x - الأدنى) / (الأعلى - الأدنى) -> النطاق [0, 1]
scaler_minmax = MinMaxScaler()
df_scaled_minmax = df.copy()
df_scaled_minmax[numeric_cols] = scaler_minmax.fit_transform(df[numeric_cols])
print("\nAfter Min-Max Normalization (range 0-1):")
print("بعد التطبيع بالأدنى-الأعلى (النطاق 0-1):")
print(f"Age - Min: {df_scaled_minmax['age'].min():.4f}, Max: {df_scaled_minmax['age'].max():.4f}")
print(f"Salary - Min: {df_scaled_minmax['salary'].min():.4f}, Max: {df_scaled_minmax['salary'].max():.4f}")
print(df_scaled_minmax[numeric_cols].head())


After Min-Max Normalization (range 0-1):
بعد التطبيع بالأدنى-الأعلى (النطاق 0-1):
Age - Min: 0.0000, Max: 1.0000
Salary - Min: 0.0000, Max: 1.0000
        age    salary  experience
0  0.974359  0.507795    0.664901
1  0.717949  0.915484    0.824535
2  0.358974  0.156025    0.558521
3  0.179487  0.352858    0.532582
4  0.512821  0.114622    0.240399


In [8]:
# 4. Label Encoding (for ordinal categories)
# الترميز بالعلامات (للفئات الترتيبية)
print("\n" + "=" * 60)
print("4. Label Encoding")
print("الترميز بالعلامات")
print("=" * 60)
label_encoder = LabelEncoder()


4. Label Encoding
الترميز بالعلامات


In [9]:
print("\n" + "=" * 60)
print("4. Label Encoding")
print("الترميز بالعلامات")
print("=" * 60)

# Create LabelEncoder
# This converts text categories to numbers
label_encoder = LabelEncoder()
print("   ✅ LabelEncoder created")



4. Label Encoding
الترميز بالعلامات
   ✅ LabelEncoder created


In [10]:
# Encode education (if we assume ordinal: Bachelor < Master < PhD)
df_encoded = df.copy()
df_encoded['education_encoded'] = label_encoder.fit_transform(df['education'])
print("\nLabel Encoding for Education:")
print("الترميز بالعلامات للتعليم:")
print("Mapping / التعيين:")
for i, label in enumerate(label_encoder.classes_):
    print(f"  {label}: {i}")
print("\nEncoded values:")
print(df_encoded[['education', 'education_encoded']].head(10))


Label Encoding for Education:
الترميز بالعلامات للتعليم:
Mapping / التعيين:
  Bachelor: 0
  Master: 1
  PhD: 2

Encoded values:
  education  education_encoded
0  Bachelor                  0
1    Master                  1
2    Master                  1
3    Master                  1
4  Bachelor                  0
5    Master                  1
6    Master                  1
7  Bachelor                  0
8       PhD                  2
9  Bachelor                  0


In [11]:
# 5. One-Hot Encoding (for nominal categories)
# الترميز الواحد-الساخن (للفئات الاسمية)
print("\n" + "=" * 60)
print("5. One-Hot Encoding")
print("الترميز الواحد-الساخن")
print("=" * 60)


5. One-Hot Encoding
الترميز الواحد-الساخن


In [12]:
print("\n" + "=" * 60)
print("5. One-Hot Encoding")
print("الترميز الواحد-الساخن")
print("=" * 60)



5. One-Hot Encoding
الترميز الواحد-الساخن


In [13]:
# Using pandas get_dummies (easier)
# استخدام pandas get_dummies (أسهل)
df_onehot = pd.get_dummies(df, columns=['city', 'education'], prefix=['city', 'edu'])
print("\nOne-Hot Encoded columns:")
print("الأعمدة المشفرة بالواحد-الساخن:")
print(df_onehot.columns.tolist())
print("\nSample of One-Hot Encoded data:")
print(df_onehot[['age', 'salary'] + [col for col in df_onehot.columns if 'city_' in col or 'edu_' in col]].head())


One-Hot Encoded columns:
الأعمدة المشفرة بالواحد-الساخن:
['age', 'salary', 'experience', 'target', 'city_Dammam', 'city_Jeddah', 'city_Khobar', 'city_Riyadh', 'edu_Bachelor', 'edu_Master', 'edu_PhD']

Sample of One-Hot Encoded data:
   age  salary  city_Dammam  city_Jeddah  city_Khobar  city_Riyadh  \
0   58   65222        False         True        False        False   
1   48   93335         True        False        False        False   
2   34   40965        False        False        False         True   
3   27   54538        False         True        False        False   
4   40   38110         True        False        False        False   

   edu_Bachelor  edu_Master  edu_PhD  
0          True       False    False  
1         False        True    False  
2         False        True    False  
3         False        True    False  
4          True       False    False  


In [14]:
# Using sklearn OneHotEncoder (for more control)
# استخدام sklearn OneHotEncoder (لمزيد من التحكم)
print("\n--- Using sklearn OneHotEncoder ---")
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
city_encoded = onehot_encoder.fit_transform(df[['city']])
print("City encoding classes:")
print(onehot_encoder.categories_)
print(f"\nEncoded shape: {city_encoded.shape}")
print("First 5 encoded values:")
print(city_encoded[:5])


--- Using sklearn OneHotEncoder ---
City encoding classes:
[array(['Dammam', 'Jeddah', 'Khobar', 'Riyadh'], dtype=object)]

Encoded shape: (100, 3)
First 5 encoded values:
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 0.]]


In [15]:
# Method 2: Using sklearn OneHotEncoder (for more control)
# Why sklearn? More control - can drop first column to avoid multicollinearity
# drop='first' removes one column (redundant - can be inferred from others)

print("\n--- Using sklearn OneHotEncoder ---")
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
city_encoded = onehot_encoder.fit_transform(df[['city']])

print("   City encoding classes:")
print(f"   {onehot_encoder.categories_}")
print(f"\n   Encoded shape: {city_encoded.shape}")
print(f"   (4 cities → 3 columns, because drop='first' removes one)")
print("\n   First 5 encoded values:")
print(city_encoded[:5])
print("\n   ✅ Each row has 1 in one column (the city), 0 in others")



--- Using sklearn OneHotEncoder ---
   City encoding classes:
   [array(['Dammam', 'Jeddah', 'Khobar', 'Riyadh'], dtype=object)]

   Encoded shape: (100, 3)
   (4 cities → 3 columns, because drop='first' removes one)

   First 5 encoded values:
[[1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 0.]]

   ✅ Each row has 1 in one column (the city), 0 in others


## Step 6: Train-Test Split | الخطوة 6: تقسيم البيانات إلى تدريب واختبار

**BEFORE**: We have all our data together. We can't evaluate model performance on data it has seen!

**AFTER**: We'll split data into training (80%) and testing (20%) sets!

**Why Train-Test Split?**
- **Training set**: Used to train/learn the model
- **Test set**: Used to evaluate model performance (unseen data)
- **Why separate?** Testing on training data gives false high scores (overfitting)
- **Standard split**: 80% train, 20% test (can vary based on dataset size)

In [16]:
# Prepare features and target
X = df[['age', 'salary', 'experience']]
y = df['target']

In [17]:
# Split into train and test sets (80% train, 20% test)
# التقسيم إلى مجموعات تدريب واختبار (80% تدريب، 20% اختبار)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class distribution
)
print(f"\nOriginal data shape: {X.shape}")
print(f"شكل البيانات الأصلية: {X.shape}")
print(f"\nTraining set:")
print(f"مجموعة التدريب:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"\nTest set:")
print(f"مجموعة الاختبار:")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"\nTrain percentage: {len(X_train) / len(X) * 100:.1f}%")
print(f"Test percentage: {len(X_test) / len(X) * 100:.1f}%")


Original data shape: (100, 3)
شكل البيانات الأصلية: (100, 3)

Training set:
مجموعة التدريب:
  X_train: (80, 3)
  y_train: (80,)

Test set:
مجموعة الاختبار:
  X_test: (20, 3)
  y_test: (20,)

Train percentage: 80.0%
Test percentage: 20.0%


## 7. Complete Preprocessing Pipeline Example

In [18]:
# مثال خط أنابيب المعالجة الكاملة
print("\n" + "=" * 60)
print("7. Complete Preprocessing Pipeline")
print("خط أنابيب المعالجة الكاملة")
print("=" * 60)

def preprocess_data(df, numeric_cols, categorical_cols, target_col, test_size=0.2):
    """
    Complete preprocessing pipeline
    خط أنابيب المعالجة الكاملة
    """
    # Separate features and target
    X = df[numeric_cols + categorical_cols].copy()
    y = df[target_col].copy()
    
    # One-hot encode categorical variables
    X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
    
    # Scale numeric features
    scaler = StandardScaler()
    X_encoded[numeric_cols] = scaler.fit_transform(X_encoded[numeric_cols])
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_encoded, y, test_size=test_size, random_state=42, stratify=y
    )
    
    return X_train, X_test, y_train, y_test, scaler


7. Complete Preprocessing Pipeline
خط أنابيب المعالجة الكاملة
